In [1]:
import csv
import pandas as pd
from transformers import RobertaTokenizerFast

#Encoder-Decoder Model
from transformers import EncoderDecoderModel
from transformers import Seq2SeqTrainer
from transformers import Trainer
from evaluate import load
#Training

from transformers import TrainingArguments
from transformers import  Seq2SeqTrainingArguments
from dataclasses import dataclass, field
from typing import Optional

from datasets import Dataset
import datasets
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss, MSELoss
bertscore = load("bertscore") 

/home2/aparna/miniconda3/envs/diffusion/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

colnames = ['source', 'target']
input_file = "train.tsv"
df = pd.read_csv(input_file, sep="\t", quoting=csv.QUOTE_NONE, encoding='utf-8', header=None, names=colnames)

In [3]:
df.head()

,source,target
0,Pterocarpus indicus ( commonly known as Amboyn...,Pterocarpus indicus ( commonly known as Amboyn...
1,"A romantic friendship , passionate friendship ...","A romantic friendship , passionate friendship ..."
2,Rocky Balboa is a 2006 American boxing sports ...,Rocky Balboa ( also known as Rocky VI ) is the...
3,Janakinath Bose ( 28 May 1860 – 1934 ) was an ...,Janakinath Bose ( 28 May 1860 – 1934 ) was an ...
4,Bionics or biologically inspired engineering i...,Bionics ( also known as biomimetics ) is the s...


In [4]:
df = df.dropna()
print(len(df))

483801


In [5]:
from #tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")
tokenizer.add_special_tokens({'additional_special_tokens': ['<sep>']})
tokenizer.add_special_tokens({'additional_special_tokens': ['<pad>']})
tokenizer.add_special_tokens({'additional_special_tokens': ['<s>']})
tokenizer.add_special_tokens({'additional_special_tokens': ['</s>']})
tokenizer.add_special_tokens({'additional_special_tokens': ['<unk>']})

maxlen = 512
def tokenize_df(df):
    target = tokenizer(df['target'], padding='max_length', truncation=True, return_tensors="pt", max_length=maxlen)
    input = tokenizer(df['source'], padding='max_length', truncation=True, return_tensors="pt", max_length=maxlen)
    input_ids = input['input_ids']
    attention_mask = input['attention_mask']
    target_iddatasets import Dataset
train_data=Dataset.from_pandas(df[:1000])
val_data=Dataset.from_pandas(df[11000:11100])
print(len(train_data))
print(len(val_data))

1000
100


In [6]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

#processing training data

In [7]:
def map_to_length(x):
  x["source_len"] = len(tokenizer(x["source"]).input_ids)
  x["source_128"] = int(x["source_len"] > 128)
  x["target_len"] = len(tokenizer(x["target"]).input_ids)
  x["target_32"] = int(x["target_len"] > 32)
  x["target_64"] = int(x["target_len"] > 64)
  return x

In [8]:
sample_size = 1000
data_stats = train_data.select(range(sample_size)).map(map_to_length, num_proc=4)
print(data_stats)

Map (num_proc=4): 100%|██████████| 1000/1000 [00:00<00:00, 3021.71 examples/s]

Dataset({
    features: ['source', 'target', 'source_len', 'source_128', 'target_len', 'target_32', 'target_64'],
    num_rows: 1000
})


In [9]:

def compute_and_print_stats(x):
  if len(x["source_len"]) == sample_size:
    print(
        "Source Mean: {}, %-sources > 128:{}, Summary Mean:{}, %-Summary > 32:{}, %-Summary > 64:{}".format(
            sum(x["source_len"]) / sample_size,
            sum(x["source_128"]) / sample_size, 
            sum(x["target_len"]) / sample_size,
            sum(x["target_32"]) / sample_size,
            sum(x["target_64"]) / sample_size,
        )
    )

output = data_stats.map(
  compute_and_print_stats, 
  batched=True,
  batch_size=-1,
)

Map: 100%|██████████| 1000/1000 [00:00<00:00, 205431.94 examples/s]

Source Mean: 37.218, %-sources > 128:0.002, Summary Mean:31.608, %-Summary > 32:0.376, %-Summary > 64:0.034


In [10]:
  #make pairwise data ,paired input and output
df = pd.DataFrame()
df["source"] = train_data["source"]
df["target"] = train_data["target"]
#make source1 and source2 and take all possible combinations
df1 = df.copy()
df2 = df.copy()
df1.columns = ["source1", "target1"]
df2.columns = ["source2", "target2"]
df1["key"] = 0
df2["key"] = 0
df = pd.merge(df1, df2, on="key").drop("key", axis=1)
df = df[df["source1"] != df["source2"]]
df = df.reset_index(drop=True)
df.head()


,source1,target1,source2,target2
0,Pterocarpus indicus ( commonly known as Amboyn...,Pterocarpus indicus ( commonly known as Amboyn...,"A romantic friendship , passionate friendship ...","A romantic friendship , passionate friendship ..."
1,Pterocarpus indicus ( commonly known as Amboyn...,Pterocarpus indicus ( commonly known as Amboyn...,Rocky Balboa is a 2006 American boxing sports ...,Rocky Balboa ( also known as Rocky VI ) is the...
2,Pterocarpus indicus ( commonly known as Amboyn...,Pterocarpus indicus ( commonly known as Amboyn...,Janakinath Bose ( 28 May 1860 – 1934 ) was an ...,Janakinath Bose ( 28 May 1860 – 1934 ) was an ...
3,Pterocarpus indicus ( commonly known as Amboyn...,Pterocarpus indicus ( commonly known as Amboyn...,Bionics or biologically inspired engineering i...,Bionics ( also known as biomimetics ) is the s...
4,Pterocarpus indicus ( commonly known as Amboyn...,Pterocarpus indicus ( commonly known as Amboyn...,The Ilyushin Il-18 ( ; NATO reporting name : C...,Ilyushin Il-18 ( Russian : Илью ́ шин Ил-18 ; ...


In [11]:
train_data = Dataset.from_pandas(df)
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)
print(len(train_data_loader))

62438


In [12]:
val = pd.DataFrame()
val["source"] = val_data["source"]
val["target"] = val_data["target"]
val1 = val.copy()
val2 = val.copy()
val1.columns = ["source1", "target1"]
val2.columns = ["source2", "target2"]
val1["key"] = 0
val2["key"] = 0
val = pd.merge(val1, val2, on="key").drop("key", axis=1)
val = val[val["source1"] != val["source2"]]
val = val.reset_index(drop=True)
val.head()

val_data = Dataset.from_pandas(val)
val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=16, shuffle=True)

In [13]:
print(len(train_data))
print(len(val_data))

999000
9900


In [24]:
#parameter setting
batch_size=256  #
encoder_max_length=128
decoder_max_length=64

def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  inputs1 = tokenizer(batch["source1"], padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs1 = tokenizer(batch["target1"], padding="max_length", truncation=True, max_length=decoder_max_length)
  inputs2 = tokenizer(batch["source2"], padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs2 = tokenizer(batch["target2"], padding="max_length", truncation=True, max_length=decoder_max_length)

  batch

  batch["input_ids1"] = inputs1.input_ids
  batch["attention_mask1"] = inputs1.attention_mask
  batch["input_ids2"] = inputs2.input_ids
  batch["attention_mask2"] = inputs2.attention_mask
  batch["decoder_input_ids1"] = outputs1.input_ids
  batch["decoder_attention_mask1"] = outputs1.attention_mask
  batch["target1"] = outputs1.input_ids.copy()
  batch["decoder_input_ids2"] = outputs2.input_ids
  batch["decoder_attention_mask2"] = outputs2.attention_mask
  batch["target2"] = outputs2.input_ids.copy()

  print("input ids1",batch["input_ids1"])
  print("input ids2",batch["input_ids2"])
  print("decoder_input_ids1",batch["decoder_input_ids1"])
  print("decoder_input_ids2",batch["decoder_input_ids2"])
  print("target1",batch["target1"])
  print("target2",batch["target2"])
  
  
  
  # because RoBERTa automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
  # We have to make sure that the PAD token is ignored
  batch["target1"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["target1"]] 
  batch["target2"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["target2"]]

  return batch


In [15]:

from transformers import EncoderDecoderModel

roberta_shared = EncoderDecoderModel.from_encoder_decoder_pretrained("roberta-base", "roberta-base", tie_encoder_decoder=True,cache_dir="/scratch/aparna/roberta_cache")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.encoder.layer.5.crossattention.self.query.bias', 'roberta.encoder.layer.4.crossattention.output.LayerNorm.bias', 'roberta.encoder.layer.1.crossattention.self.value.weight', 'roberta.encoder.layer.8.crossattention.self.value.weight', 'roberta.encoder.layer.1.crossattention.output.dense.bias', 'roberta.encoder.layer.9.crossattention.output.dense.bias', 'roberta.encoder.layer.5.crossattention.self.query.weight', 'roberta.encoder.layer.4.crossattention.self.value.bias', 'roberta.encoder.layer.4.crossattention.self.key.weight', 'roberta.encoder.layer.9.crossattention.outpu

In [16]:

# set special tokens
roberta_shared.config.decoder_start_token_id = tokenizer.bos_token_id                                             
roberta_shared.config.eos_token_id = tokenizer.eos_token_id

# sensible parameters for beam search
# set decoding params                               
roberta_shared.config.max_length = 100
roberta_shared.config.early_stopping = True
roberta_shared.config.no_repeat_ngram_size = 3
roberta_shared.config.length_penalty = 2.0
roberta_shared.config.num_beams = 4
roberta_shared.config.vocab_size = roberta_shared.config.encoder.vocab_size

In [17]:
# load rouge for validation
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    print(pred)
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

/tmp/ipykernel_12664/1500687920.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = datasets.load_metric("rouge")


In [25]:

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
    
        target1 = inputs.pop("target1")
        target2 = inputs.pop("target2")
        input_ids1 = inputs.pop("input_ids1")
        attention_mask1 = inputs.pop("attention_mask1")
        input_ids2 = inputs.pop("input_ids2")
        attention_mask2 = inputs.pop("attention_mask2")
        decoder_input_ids1 = inputs.pop("decoder_input_ids1")
        decoder_attention_mask1 = inputs.pop("decoder_attention_mask1")
        decoder_input_ids2 = inputs.pop("decoder_input_ids2")
        decoder_attention_mask2 = inputs.pop("decoder_attention_mask2")

        # forward pass
        outputs1 = model(
            input_ids=input_ids1,
            attention_mask=attention_mask1,
            decoder_input_ids=decoder_input_ids1,
            decoder_attention_mask=decoder_attention_mask1,
            labels=target1,
        )
        outputs2 = model(
            input_ids=input_ids2,
            attention_mask=attention_mask2,
            decoder_input_ids=decoder_input_ids2,
            decoder_attention_mask=decoder_attention_mask2,
            labels=target2,
        )
        
        logits1 = outputs1.get("logits")
        logits2 = outputs2.get("logits")

        #conver to words
        pred_ids1 = torch.argmax(logits1, dim=-1)
        pred_str1 = tokenizer.batch_decode(pred_ids1, skip_special_tokens=True)
        inputs_str1 = tokenizer.batch_decode(inputs["input_ids1"], skip_special_tokens=True)
        pred_ids2 = torch.argmax(logits2, dim=-1)
        pred_str2 = tokenizer.batch_decode(pred_ids2, skip_special_tokens=True)
        inputs_str2 = tokenizer.batch_decode(inputs["input_ids2"], skip_special_tokens=True)

        #bert score
        F1 = bertscore(pred_str1, inputs_str1, lang="en", verbose=True).f1.mean()
        F2 = bertscore(pred_str2, inputs_str2, lang="en", verbose=True).f1.mean()
        F1 = torch.tensor(F1)
        F2 = torch.tensor(F2)
        #length penalty
        label_len1 = torch.sum(inputs["target1"] != -100, dim=-1)
        input_len1 = torch.sum(inputs["input_ids1"] != tokenizer.pad_token_id, dim=-1)
        compression_ratio1 = input_len1 / label_len1

        pred_len1 = torch.sum(pred_ids1 != tokenizer.pad_token_id, dim=-1)
        pred_ratio1 = pred_len1 / label_len1

        label_len2 = torch.sum(inputs["target2"] != -100, dim=-1)
        input_len2 = torch.sum(inputs["input_ids2"] != tokenizer.pad_token_id, dim=-1)
        compression_ratio2 = input_len2 / label_len2

        pred_len2 = torch.sum(pred_ids2 != tokenizer.pad_token_id, dim=-1)
        pred_ratio2 = pred_len2 / label_len2

        len_penalty1 = torch.exp(-torch.abs(compression_ratio1 - pred_ratio1))
        len_penalty2 = torch.exp(-torch.abs(compression_ratio2 - pred_ratio2))

        l = torch.abs(len_penalty2*F1 - len_penalty1*F2)
        #jacard similarity loss
        intersection1 = torch.sum((pred_ids1 == inputs["target1"]) & (inputs["target1"] != -100), dim=-1)
        union1 = torch.sum((pred_ids1 != tokenizer.pad_token_id) | (inputs["target1"] != -100), dim=-1)
        jacard1 = intersection1 / union1

        intersection2 = torch.sum((pred_ids2 == inputs["target2"]) & (inputs["target2"] != -100), dim=-1)
        union2 = torch.sum((pred_ids2 != tokenizer.pad_token_id) | (inputs["target2"] != -100), dim=-1)
        jacard2 = intersection2 / union2

        d = torch.abs(jacard2 - jacard1)
        loss = max(0,1 -l*d)




        return (loss, outputs1,outputs2) if return_outputs else loss

In [19]:
# training_args = Seq2SeqTrainingArguments(
#     output_dir="/scratch/aparna/",
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     predict_with_generate=True,
#     do_train=True,
#     do_eval=True,
#     logging_steps=2, 
#     save_steps=16, 
#     eval_steps=500, 
#     warmup_steps=500, 
#     overwrite_output_dir=True,
#     save_total_limit=1,
#     fp16=True, 

# )

# # instantiate trainer
# trainer = CustomTrainer(
#     model=roberta_shared,
#     args=training_args,
#     compute_metrics=compute_metrics,
#     train_dataset=train_data.map(process_data_to_model_inputs, batched=True),
#     eval_dataset=val_data.map(process_data_to_model_inputs, batched=True),
#     tokenizer=tokenizer,
# )


In [20]:
# trainer.train()

In [26]:
def compute_loss( model, inputs, return_outputs=False):
    
        target1 = inputs.pop("target1")
        target2 = inputs.pop("target2")
        input_ids1 = inputs.pop("input_ids1")
        attention_mask1 = inputs.pop("attention_mask1")
        input_ids2 = inputs.pop("input_ids2")
        attention_mask2 = inputs.pop("attention_mask2")
        decoder_input_ids1 = inputs.pop("decoder_input_ids1")
        decoder_attention_mask1 = inputs.pop("decoder_attention_mask1")
        decoder_input_ids2 = inputs.pop("decoder_input_ids2")
        decoder_attention_mask2 = inputs.pop("decoder_attention_mask2")

        # forward pass
        outputs1 = model(
            input_ids=torch.FloatTensor(input_ids1),
            attention_mask=torch.FloatTensor(attention_mask1),
            decoder_input_ids=torch.FloatTensor(decoder_input_ids1),
            decoder_attention_mask=torch.FloatTensor(decoder_attention_mask1),
            labels=torch.FloatTensor(target1),
        )
        outputs2 = model(
            input_ids=input_ids2,
            attention_mask=attention_mask2,
            decoder_input_ids=decoder_input_ids2,
            decoder_attention_mask=decoder_attention_mask2,
            labels=target2,
        )
        
        logits1 = outputs1.get("logits")
        logits2 = outputs2.get("logits")

        #conver to words
        pred_ids1 = torch.argmax(logits1, dim=-1)
        pred_str1 = tokenizer.batch_decode(pred_ids1, skip_special_tokens=True)
        inputs_str1 = tokenizer.batch_decode(inputs["input_ids1"], skip_special_tokens=True)
        pred_ids2 = torch.argmax(logits2, dim=-1)
        pred_str2 = tokenizer.batch_decode(pred_ids2, skip_special_tokens=True)
        inputs_str2 = tokenizer.batch_decode(inputs["input_ids2"], skip_special_tokens=True)

        #bert score
        F1 = bertscore(pred_str1, inputs_str1, lang="en", verbose=True).f1.mean()
        F2 = bertscore(pred_str2, inputs_str2, lang="en", verbose=True).f1.mean()
        F1 = torch.tensor(F1)
        F2 = torch.tensor(F2)
        #length penalty
        label_len1 = torch.sum(inputs["target1"] != -100, dim=-1)
        input_len1 = torch.sum(inputs["input_ids1"] != tokenizer.pad_token_id, dim=-1)
        compression_ratio1 = input_len1 / label_len1

        pred_len1 = torch.sum(pred_ids1 != tokenizer.pad_token_id, dim=-1)
        pred_ratio1 = pred_len1 / label_len1

        label_len2 = torch.sum(inputs["target2"] != -100, dim=-1)
        input_len2 = torch.sum(inputs["input_ids2"] != tokenizer.pad_token_id, dim=-1)
        compression_ratio2 = input_len2 / label_len2

        pred_len2 = torch.sum(pred_ids2 != tokenizer.pad_token_id, dim=-1)
        pred_ratio2 = pred_len2 / label_len2

        len_penalty1 = torch.exp(-torch.abs(compression_ratio1 - pred_ratio1))
        len_penalty2 = torch.exp(-torch.abs(compression_ratio2 - pred_ratio2))

        l = torch.abs(len_penalty2*F1 - len_penalty1*F2)
        #jacard similarity loss
        intersection1 = torch.sum((pred_ids1 == inputs["target1"]) & (inputs["target1"] != -100), dim=-1)
        union1 = torch.sum((pred_ids1 != tokenizer.pad_token_id) | (inputs["target1"] != -100), dim=-1)
        jacard1 = intersection1 / union1

        intersection2 = torch.sum((pred_ids2 == inputs["target2"]) & (inputs["target2"] != -100), dim=-1)
        union2 = torch.sum((pred_ids2 != tokenizer.pad_token_id) | (inputs["target2"] != -100), dim=-1)
        jacard2 = intersection2 / union2

        d = torch.abs(jacard2 - jacard1)
        loss = max(0,1 -l*d)




        return (loss, outputs1,outputs2) if return_outputs else loss

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def train(model, iterator, optimizer, criterion):
    model = model.to(device)
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(iterator):
        optimizer.zero_grad()
        print(batch)
       # process batch
        batch = process_data_to_model_inputs(batch)
        print(batch)
        loss = compute_loss(model, batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        if i % 100 == 0:
            print(f"Train Loss: {loss.item()}")

    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            batch = process_data_to_model_inputs(batch)
            print(batch)
            loss = compute_loss(model, batch)
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

optimizer = torch.optim.Adam(roberta_shared.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

N_EPOCHS = 10
best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):
    train_loss = train(roberta_shared, train_data_loader, optimizer, criterion)
    valid_loss = evaluate(roberta_shared, val_data_loader, criterion)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(roberta_shared.state_dict(), '/scratch/aparna/model.pt')
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f}')



{'source1': ['The Zygnemataceae are a family of filamentous or unicellular , uniseriate ( unbranched ) green algae .', 'Murdo John MacKay ( born August 8 , 1917 in Fort William , Ontario - d . August 8 , 2000 ) is a Canadian retired professional ice hockey forward .', 'David IV , also known as David the Builder ( Georgian : , " " ) ( 1073 – 24 January 1125 ) , of the Bagrationi dynasty , was a king of Georgia from 1089 until his death in 1125 .', 'Cutout animation is a form of stop-motion animation using flat characters , props and backgrounds cut from materials such as paper , card , stiff fabric or even photographs .', 'BBC Radio 6 Music ( also known as BBC 6 Music or BBC 6 ) is a digital radio station run by the BBC , specialising primarily in alternative music .', 'There are over 100 museums in Seoul .', 'Calvin Klein Inc. is an American luxury fashion house established in 1968 .', 'Robie Creek is a census-designated place in Boise County , Idaho , United States .', 'The Iveco Dail

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)